In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
path = os.path.join("../../data/afras/raw", "integrated_data_20251106_213536.csv")


ValueError: Key backend: 'module://matplotlib_inline.backend_inline' is not a valid value for backend; supported values are ['gtk3agg', 'gtk3cairo', 'gtk4agg', 'gtk4cairo', 'macosx', 'nbagg', 'notebook', 'qtagg', 'qtcairo', 'qt5agg', 'qt5cairo', 'tkagg', 'tkcairo', 'webagg', 'wx', 'wxagg', 'wxcairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']

2. Data Loading, Preparing and Scaling

In [67]:
data = pd.read_csv(path)

#data['Date'] = pd.to_datetime(data['Date'])
#data.set_index('Date', inplace=True)

env_data_cols = ['env0', 'env1', 'env2', 'env3']
finger_cols = ['thumb_tip','thumb_base','index','middle','ring','pinky']

data[env_data_cols] = data[env_data_cols].astype(float)
data[finger_cols] = data[finger_cols].astype(float)

scaler = MinMaxScaler(feature_range=(0, 1)) #Scales data to be between 0 and 1
env_data_scaled = scaler.fit_transform(data[env_data_cols])
finger_scaled = scaler.fit_transform(data[finger_cols])


3. Creating Sequences and Train-Test Split

In [ ]:
window_size = 100 # Number of time steps to look back

def create_emg_sequences(env_data_scaled, finger_scaled, window_size):
    X, y = [], []
    for i in range(len(env_data_scaled) - window_size):
        X.append(env_data_scaled[i:(i + window_size), :])
        y.append(finger_scaled[i + window_size - 1])  # current, not future
    return np.array(X), np.array(y)

X, y = create_emg_sequences(env_data_scaled, finger_scaled, window_size)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False  # How much % of the data we test on
)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (40108, 100, 4)
y_train shape: (40108, 6)


In [69]:
num_features = X_train.shape[2]   # = 4 (env0–env3)
num_outputs = y_train.shape[1]    # = 6 (finger columns)

#model = Sequential()
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(window_size, num_features)),
    Dropout(0.3),
    LSTM(64, return_sequences=False),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_outputs, activation='sigmoid')
])

"""


# First LSTM layer (returns sequences so the next LSTM can process them)
model.add(LSTM(units=128, return_sequences=True, input_shape=(window_size, num_features)))
model.add(Dropout(0.2))

# Second LSTM layer (final output sequence)
model.add(LSTM(units=128))
model.add(Dropout(0.2))

# Dense output layer for 6 finger values
model.add(Dense(num_outputs, activation='sigmoid'))  # sigmoid to stay in [0,1] range
"""
# Compile the model for regression
model.compile(optimizer='adam', loss='mean_squared_error')

model.summary()

c:\Users\elvin\Grip\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_25 (LSTM)                  │ (None, 100, 128)       │        68,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_26 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,054 (476.77 KB)

 Trainable params: 122,054 (476.77 KB)

 Non-trainable params: 0 (0.00 B)

In [70]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    shuffle=True
)

# Predict
predictions = model.predict(X_test)

# Inverse transform finger scaling (NOT the EMG scaling)
predictions_inv = scaler.inverse_transform(predictions)
y_test_inv = scaler.inverse_transform(y_test)

# Compute RMSE for each finger
rmse_per_finger = np.sqrt(np.mean((y_test_inv - predictions_inv) ** 2, axis=0))

# Print RMSE for each finger
for i, col in enumerate(finger_cols):
    print(f"RMSE ({col}): {rmse_per_finger[i]:.4f}")

# Optionally, overall RMSE across all fingers
overall_rmse = np.sqrt(np.mean((y_test_inv - predictions_inv) ** 2))
print(f"Overall RMSE: {overall_rmse:.4f}")


Epoch 1/100
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 73s 61ms/step - loss: 0.0705 - val_loss: 0.0970
Epoch 2/100
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 70s 62ms/step - loss: 0.0530 - val_loss: 0.0928
Epoch 3/100
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 79s 70ms/step - loss: 0.0492 - val_loss: 0.0875
Epoch 4/100
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 72s 63ms/step - loss: 0.0458 - val_loss: 0.0907
Epoch 5/100
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 99s 88ms/step - loss: 0.0437 - val_loss: 0.0897
Epoch 6/100
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 94s 83ms/step - loss: 0.0422 - val_loss: 0.0926
Epoch 7/100
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 95s 84ms/step - loss: 0.0401 - val_loss: 0.1003
Epoch 8/100
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 79s 70ms/step - loss: 0.0400 - val_loss: 0.0972
Epoch 9/100
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 100s 89ms/step - loss: 0.0372 - val_loss: 0.0957
Epoch 10/100
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 93s 83ms/step - loss: 0.0379 - val_loss: 0.1011
Epoch 11/100
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 92s 81ms/step - loss: 0.0362 - val_loss: 0.0

In [ ]:
plt.figure(figsize=(15, 10))
for i, col in enumerate(finger_cols):
    plt.subplot(3, 2, i + 1)  # 3 rows x 2 cols grid
    plt.plot(y_test_inv[:, i], label='Actual')
    plt.plot(predictions_inv[:, i], label='Predicted')
    plt.title(f'Finger: {col}')
    plt.xlabel('Time step')
    plt.ylabel('Normalized position')
    plt.legend()
    plt.xlim(9000, 9100)

plt.tight_layout()
plt.show()
print("Done")

NameError: name 'plt' is not defined